In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM , AutoTokenizer

dataset = load_dataset("wmt16" , "de-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/282M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/267M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/277M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/475k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_data(examples):
  inputs = [f"Translate From English to German: {example['en']}" for example in examples["translation"]]
  targets = [example['de'] for example in examples['translation']]

  model_inputs = tokenizer(inputs , max_length=128 , truncation=True , padding="max_length" , return_tensors='tf')
  labels = tokenizer(targets  , max_length=128 , truncation=True , padding ="max_length" ,return_tensors="tf" ).input_ids
  model_inputs["labels"]=labels
  decoder_inputs = tokenizer(targets , max_length=128 , truncation=True , padding="max_length")
  model_inputs["decoder_input_ids"] = decoder_inputs["input_ids"]

  return model_inputs

In [ ]:
train_dataset = dataset["train"].select(range(20000)).map(preprocess_data , batched=True)
test_dataset = dataset["test"].select(range(1000)).map(preprocess_data , batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.to_tf_dataset(
    columns=["input_ids" , "attention_mask" , "decoder_input_ids"],
    label_cols =['labels'],
    shuffle=True ,
    batch_size=256,
    collate_fn=None
)
test_dataset = test_dataset.to_tf_dataset(
    columns= ["input_ids" , "attention_mask" , "decoder_input_ids"],
    shuffle=True,
    batch_size =256 ,
    collate_fn=None
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [ ]:
from transformers import TFAutoModelForSeq2SeqLM , AutoTokenizer
from tensorflow.keras.layers import Dense
import tensorflow as tf

model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
class LoRALayer(tf.keras.layers.Layer):
  def __init__(self , dense , rank=4):
    super().__init__()
    self.dense =dense
    self.rank = rank

  def build(self , input_shape):
    self.w_a = self.add_weight(shape=(input_shape[-1] ,self.rank),
                               initializer="random_normal" ,
                               trainable=True  ,
                               name='w_a')
    self.w_b = self.add_weight(shape=(self.rank , self.dense.units) ,
                               initializer="random_normanl" ,
                               trainable=True , name="w_b")

  def call(self , inputs):
    original_output = self.dense(inputs)
    lora_output = tf.matmul(tf.matmul(inputs , self.w_a ) ,self.w_b)
    self.dense.trainable=False
    return original_output + lora_output

In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247577856 (944.43 MB)
Trainable params: 247577856 (944.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import tf_keras

for ix ,layer in enumerate(model.decoder._flatten_layers()):
  if isinstance(layer , tf_keras.src.layers.core.dense.Dense):
    layer.trainable=False
    layer = LoRALayer(layer)
  else:
    layer.trainable=False
  model.get_layer("encoder").trainable =False
  model.get_layer("shared").trainable =False

  model.layers[3] = LoRALayer(model.get_layer('lm_head'))

In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247577856 (944.43 MB)
Trainable params: 24674304 (94.12 MB)
Non-trainable params: 222903552 (850.31 MB)
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=1e-3),
              loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [34]:
model.fit(train_dataset , validation_data=test_dataset , epochs=3)